# 🏦 Notebook 03: Silver Layer - Slowly Changing Dimensions (SCD)

## Implementación de SCD Type 1 y Type 2 con Delta Lake

---

### Objetivos de este notebook:
1. Entender los diferentes tipos de SCD (1, 2 y 3)
2. Implementar SCD Type 1 (Sobrescritura)
3. Implementar SCD Type 2 (Historial Completo)
4. Implementar SCD Híbrido (Type 1 + Type 2)
5. Procesar datos desde Bronze usando CDC

### Conceptos clave:

| Tipo SCD | Descripción | Caso de Uso |
|----------|-------------|-------------|
| **Type 0** | Sin cambios (valores constantes) | IDs, códigos fijos |
| **Type 1** | Sobrescribir valor anterior | Corrección de errores, datos no históricos |
| **Type 2** | Mantener historial completo | Análisis histórico, auditoría, compliance |
| **Type 3** | Mantener valor anterior y actual | Comparación simple antes/después |

---

## 1️⃣ Configuración Inicial

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
import uuid

# Configuración
DATABASE_NAME = "financial_lakehouse"
spark.sql(f"USE {DATABASE_NAME}")

# Tablas Bronze (fuente)
BRONZE_CLIENTES = "bronze_clientes"
BRONZE_CUENTAS = "bronze_cuentas"

# Tablas Silver (destino)
SILVER_DIM_CLIENTES = "silver_dim_clientes"  # SCD Type 2
SILVER_DIM_CUENTAS = "silver_dim_cuentas"    # SCD Type 1 + Type 2 híbrido

print(f"✅ Configuración cargada para {DATABASE_NAME}")

## 2️⃣ Crear Tablas Silver con Estructura SCD

In [ ]:
# ============================================
# TABLA SILVER: DIMENSIÓN CLIENTES (SCD TYPE 2)
# ============================================

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {SILVER_DIM_CLIENTES} (
        -- Surrogate Key (generada automáticamente)
        cliente_key BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Surrogate key para la dimensión',
        
        -- Business Key (Natural Key del sistema origen)
        cliente_id STRING NOT NULL COMMENT 'Business key - ID original del cliente',
        
        -- Atributos del cliente
        nombre STRING COMMENT 'Nombre completo',
        email STRING COMMENT 'Email del cliente',
        telefono STRING COMMENT 'Teléfono',
        direccion STRING COMMENT 'Dirección completa',
        ciudad STRING COMMENT 'Ciudad',
        pais STRING COMMENT 'País',
        codigo_postal STRING COMMENT 'Código postal',
        fecha_nacimiento DATE COMMENT 'Fecha de nacimiento',
        genero STRING COMMENT 'Género',
        segmento_cliente STRING COMMENT 'Segmento: RETAIL, PREMIUM, VIP, CORPORATE',
        estado STRING COMMENT 'Estado: ACTIVO, INACTIVO, SUSPENDIDO',
        
        -- Columnas SCD Type 2
        fecha_inicio TIMESTAMP NOT NULL COMMENT 'Fecha de inicio de vigencia del registro',
        fecha_fin TIMESTAMP COMMENT 'Fecha de fin de vigencia (NULL = registro actual)',
        es_actual BOOLEAN NOT NULL COMMENT 'Flag que indica si es el registro vigente',
        version INT NOT NULL COMMENT 'Número de versión del registro',
        
        -- Hash para detección de cambios
        hash_atributos STRING COMMENT 'Hash MD5 de atributos tracked para SCD2',
        
        -- Metadatos de auditoría
        fecha_creacion TIMESTAMP COMMENT 'Fecha de creación del registro en Silver',
        fecha_actualizacion TIMESTAMP COMMENT 'Última actualización del registro',
        fuente_origen STRING COMMENT 'Sistema origen de los datos'
    )
    USING DELTA
    CLUSTER BY (cliente_id, es_actual)
    COMMENT 'Dimensión de Clientes con SCD Type 2 - Historial completo de cambios'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.columnMapping.mode' = 'name'
    )
""")

print(f"✅ Tabla {SILVER_DIM_CLIENTES} creada con estructura SCD Type 2")

In [ ]:
# ============================================
# TABLA SILVER: DIMENSIÓN CUENTAS (SCD HÍBRIDO)
# ============================================
# SCD Type 1: saldo_actual, saldo_disponible (sobrescribir)
# SCD Type 2: limite_credito, estado, tipo_cuenta (historial)

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {SILVER_DIM_CUENTAS} (
        -- Surrogate Key
        cuenta_key BIGINT GENERATED ALWAYS AS IDENTITY COMMENT 'Surrogate key',
        
        -- Business Keys
        numero_cuenta STRING NOT NULL COMMENT 'Número de cuenta único',
        cliente_id STRING NOT NULL COMMENT 'FK al cliente',
        
        -- Atributos SCD Type 2 (generan nueva versión)
        tipo_cuenta STRING COMMENT 'Tipo: AHORRO, CORRIENTE, CREDITO, INVERSION',
        limite_credito DECIMAL(18,2) COMMENT 'Límite de crédito',
        estado STRING COMMENT 'Estado: ACTIVA, BLOQUEADA, CERRADA',
        tasa_interes DECIMAL(5,4) COMMENT 'Tasa de interés',
        sucursal STRING COMMENT 'Sucursal asignada',
        
        -- Atributos SCD Type 1 (sobrescribir sin historial)
        saldo_actual DECIMAL(18,2) COMMENT 'Saldo actual - Type 1',
        saldo_disponible DECIMAL(18,2) COMMENT 'Saldo disponible - Type 1',
        fecha_ultimo_movimiento TIMESTAMP COMMENT 'Último movimiento - Type 1',
        
        -- Atributos estáticos (no cambian)
        moneda STRING COMMENT 'Código de moneda',
        fecha_apertura DATE COMMENT 'Fecha de apertura original',
        ejecutivo_cuenta STRING COMMENT 'Ejecutivo asignado',
        
        -- Columnas SCD Type 2
        fecha_inicio TIMESTAMP NOT NULL,
        fecha_fin TIMESTAMP,
        es_actual BOOLEAN NOT NULL,
        version INT NOT NULL,
        
        -- Hash para SCD2 (solo columnas tracked)
        hash_scd2 STRING COMMENT 'Hash de columnas SCD Type 2',
        
        -- Metadatos
        fecha_creacion TIMESTAMP,
        fecha_actualizacion TIMESTAMP
    )
    USING DELTA
    CLUSTER BY (numero_cuenta, cliente_id, es_actual)
    COMMENT 'Dimensión de Cuentas con SCD Híbrido (Type 1 + Type 2)'
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true'
    )
""")

print(f"✅ Tabla {SILVER_DIM_CUENTAS} creada con estructura SCD Híbrida")

## 3️⃣ Implementación SCD Type 2 para Clientes

### Lógica:
1. **Registro nuevo**: INSERT con `es_actual = true`, `version = 1`
2. **Registro existente con cambios**: 
   - UPDATE registro anterior: `es_actual = false`, `fecha_fin = now()`
   - INSERT nuevo registro: `es_actual = true`, `version = version + 1`
3. **Registro existente sin cambios**: No hacer nada

In [ ]:
# ============================================
# FUNCIÓN: CALCULAR HASH DE ATRIBUTOS SCD2
# ============================================

def calculate_scd2_hash(df, columns_to_track):
    """
    Calcula un hash MD5 de las columnas que se rastrean para SCD Type 2.
    Esto permite detectar cambios eficientemente.
    
    Args:
        df: DataFrame fuente
        columns_to_track: Lista de columnas a incluir en el hash
        
    Returns:
        DataFrame con columna 'hash_atributos' añadida
    """
    # Concatenar columnas y calcular MD5
    concat_cols = concat_ws("||", *[coalesce(col(c).cast("string"), lit("NULL")) for c in columns_to_track])
    return df.withColumn("hash_atributos", md5(concat_cols))

# Definir columnas a trackear para SCD2 en Clientes
CLIENTES_SCD2_COLUMNS = [
    "nombre",
    "direccion",
    "ciudad",
    "pais",
    "codigo_postal",
    "segmento_cliente",
    "estado"
]

print(f"📋 Columnas tracked para SCD2 en Clientes: {CLIENTES_SCD2_COLUMNS}")

In [ ]:
# ============================================
# FUNCIÓN: APLICAR SCD TYPE 2
# ============================================

def apply_scd_type2(
    source_df,
    target_table: str,
    business_key: list,
    scd2_columns: list,
    static_columns: list = None
):
    """
    Aplica lógica SCD Type 2 usando MERGE de Delta Lake.
    
    Args:
        source_df: DataFrame con datos nuevos/actualizados
        target_table: Nombre de la tabla destino
        business_key: Lista de columnas que forman la clave de negocio
        scd2_columns: Columnas que generan nueva versión al cambiar
        static_columns: Columnas que nunca cambian
    """
    from delta.tables import DeltaTable
    
    # Añadir hash de atributos SCD2
    source_with_hash = calculate_scd2_hash(source_df, scd2_columns)
    
    # Verificar si la tabla destino tiene datos
    target_count = spark.table(target_table).count()
    
    if target_count == 0:
        # Primera carga: INSERT directo
        print(f"📥 Carga inicial en {target_table}")
        
        initial_load = source_with_hash.select(
            *[col(c) for c in source_df.columns if c not in ['fecha_ingesta', 'operacion', 'fuente']],
            current_timestamp().alias("fecha_inicio"),
            lit(None).cast("timestamp").alias("fecha_fin"),
            lit(True).alias("es_actual"),
            lit(1).alias("version"),
            col("hash_atributos"),
            current_timestamp().alias("fecha_creacion"),
            current_timestamp().alias("fecha_actualizacion"),
            col("fuente").alias("fuente_origen")
        )
        
        initial_load.write.format("delta").mode("append").saveAsTable(target_table)
        print(f"✅ Cargados {initial_load.count()} registros iniciales")
        return
    
    # Merge para actualizaciones incrementales
    print(f"🔄 Procesando cambios incrementales en {target_table}")
    
    # Obtener tabla destino como DeltaTable
    target = DeltaTable.forName(spark, target_table)
    
    # Construir condición de merge (business key + es_actual)
    merge_condition = " AND ".join([
        f"target.{k} = source.{k}" for k in business_key
    ]) + " AND target.es_actual = true"
    
    # Condición para detectar cambios (hash diferente)
    update_condition = "target.hash_atributos <> source.hash_atributos"
    
    # Preparar source con columnas adicionales
    source_prepared = source_with_hash.alias("source")
    
    # Ejecutar MERGE
    target.alias("target").merge(
        source_prepared,
        merge_condition
    ).whenMatchedUpdate(
        # Cuando hay match Y el hash cambió: cerrar registro anterior
        condition=update_condition,
        set={
            "fecha_fin": current_timestamp(),
            "es_actual": lit(False),
            "fecha_actualizacion": current_timestamp()
        }
    ).whenNotMatchedInsert(
        # Registro completamente nuevo
        values={
            **{k: col(f"source.{k}") for k in business_key},
            **{c: col(f"source.{c}") for c in scd2_columns},
            "email": col("source.email"),
            "telefono": col("source.telefono"),
            "fecha_nacimiento": col("source.fecha_nacimiento"),
            "genero": col("source.genero"),
            "fecha_inicio": current_timestamp(),
            "fecha_fin": lit(None),
            "es_actual": lit(True),
            "version": lit(1),
            "hash_atributos": col("source.hash_atributos"),
            "fecha_creacion": current_timestamp(),
            "fecha_actualizacion": current_timestamp(),
            "fuente_origen": col("source.fuente")
        }
    ).execute()
    
    print("✅ MERGE completado")
    
    # Ahora insertar nuevas versiones para los registros que cambiaron
    # Identificar registros que fueron marcados como no actuales
    closed_records = spark.table(target_table).filter(
        (col("es_actual") == False) & 
        (col("fecha_fin").isNotNull()) &
        (col("fecha_fin") > date_sub(current_timestamp(), 1))  # Cerrados recientemente
    ).select(
        *business_key,
        col("version")
    )
    
    # Join con source para insertar nuevas versiones
    new_versions = source_with_hash.alias("src").join(
        closed_records.alias("closed"),
        on=business_key,
        how="inner"
    ).select(
        *[col(f"src.{c}") for c in source_df.columns if c not in ['fecha_ingesta', 'operacion', 'fuente']],
        current_timestamp().alias("fecha_inicio"),
        lit(None).cast("timestamp").alias("fecha_fin"),
        lit(True).alias("es_actual"),
        (col("closed.version") + 1).alias("version"),
        col("src.hash_atributos"),
        current_timestamp().alias("fecha_creacion"),
        current_timestamp().alias("fecha_actualizacion"),
        col("src.fuente").alias("fuente_origen")
    )
    
    if new_versions.count() > 0:
        new_versions.write.format("delta").mode("append").saveAsTable(target_table)
        print(f"✅ Insertadas {new_versions.count()} nuevas versiones")

print("✅ Función apply_scd_type2 definida")

In [ ]:
# ============================================
# CARGAR DATOS DESDE BRONZE A SILVER (SCD2)
# ============================================

# Leer datos de Bronze (usando Change Data Feed para procesar solo cambios)
bronze_clientes = spark.table(BRONZE_CLIENTES)

print(f"📊 Registros en Bronze: {bronze_clientes.count()}")

# Aplicar SCD Type 2
apply_scd_type2(
    source_df=bronze_clientes,
    target_table=SILVER_DIM_CLIENTES,
    business_key=["cliente_id"],
    scd2_columns=CLIENTES_SCD2_COLUMNS
)

In [ ]:
# Verificar resultado
print(f"\n📋 Muestra de {SILVER_DIM_CLIENTES}:")
spark.table(SILVER_DIM_CLIENTES).select(
    "cliente_key",
    "cliente_id",
    "nombre",
    "segmento_cliente",
    "ciudad",
    "es_actual",
    "version",
    "fecha_inicio",
    "fecha_fin"
).orderBy("cliente_id", "version").show(10, truncate=False)

## 4️⃣ Simular Cambios y Ver Historial SCD2

In [ ]:
# ============================================
# SIMULAR CAMBIOS EN BRONZE PARA VER SCD2
# ============================================

# Obtener un cliente existente
cliente_ejemplo = spark.table(SILVER_DIM_CLIENTES).filter(
    col("es_actual") == True
).select("cliente_id").first()["cliente_id"]

print(f"📌 Cliente ejemplo: {cliente_ejemplo}")

# Simular cambio de segmento y dirección en Bronze
spark.sql(f"""
    UPDATE {BRONZE_CLIENTES}
    SET 
        segmento_cliente = 'VIP',
        direccion = 'Av. Los Ejecutivos 1000, Piso 15',
        ciudad = 'San Isidro',
        operacion = 'UPDATE',
        fecha_ingesta = current_timestamp()
    WHERE cliente_id = '{cliente_ejemplo}'
""")

print(f"✅ Cliente {cliente_ejemplo} actualizado en Bronze")

In [ ]:
# Volver a procesar para aplicar SCD2
bronze_clientes_updated = spark.table(BRONZE_CLIENTES)

apply_scd_type2(
    source_df=bronze_clientes_updated,
    target_table=SILVER_DIM_CLIENTES,
    business_key=["cliente_id"],
    scd2_columns=CLIENTES_SCD2_COLUMNS
)

In [ ]:
# Ver historial del cliente
print(f"\n📜 Historial SCD2 para {cliente_ejemplo}:")
spark.table(SILVER_DIM_CLIENTES).filter(
    col("cliente_id") == cliente_ejemplo
).select(
    "cliente_key",
    "cliente_id",
    "nombre",
    "segmento_cliente",
    "direccion",
    "ciudad",
    "es_actual",
    "version",
    "fecha_inicio",
    "fecha_fin"
).orderBy("version").show(truncate=False)

## 5️⃣ Implementación SCD Híbrido (Type 1 + Type 2) para Cuentas

En cuentas bancarias, algunos atributos cambian frecuentemente (saldo) y no necesitan historial, mientras que otros cambios son significativos (estado, límite de crédito).

In [ ]:
# ============================================
# FUNCIÓN: APLICAR SCD HÍBRIDO (TYPE 1 + TYPE 2)
# ============================================

def apply_scd_hybrid(
    source_df,
    target_table: str,
    business_key: list,
    scd1_columns: list,  # Columnas que se sobrescriben
    scd2_columns: list,  # Columnas que generan nueva versión
    static_columns: list  # Columnas que nunca cambian
):
    """
    Aplica lógica SCD híbrida:
    - SCD Type 1: Sobrescribir valores (saldos, fechas de movimiento)
    - SCD Type 2: Nueva versión para cambios importantes (estado, límites)
    """
    
    # Hash solo de columnas SCD2
    source_with_hash = source_df.withColumn(
        "hash_scd2",
        md5(concat_ws("||", *[coalesce(col(c).cast("string"), lit("NULL")) for c in scd2_columns]))
    )
    
    target_count = spark.table(target_table).count()
    
    if target_count == 0:
        # Carga inicial
        print(f"📥 Carga inicial en {target_table}")
        
        initial_df = source_with_hash.select(
            # Business keys
            *[col(c) for c in business_key],
            # SCD2 columns
            *[col(c) for c in scd2_columns],
            # SCD1 columns
            *[col(c) for c in scd1_columns],
            # Static columns
            *[col(c) for c in static_columns],
            # SCD metadata
            current_timestamp().alias("fecha_inicio"),
            lit(None).cast("timestamp").alias("fecha_fin"),
            lit(True).alias("es_actual"),
            lit(1).alias("version"),
            col("hash_scd2"),
            current_timestamp().alias("fecha_creacion"),
            current_timestamp().alias("fecha_actualizacion")
        )
        
        initial_df.write.format("delta").mode("append").saveAsTable(target_table)
        print(f"✅ Cargados {initial_df.count()} registros")
        return
    
    # Merge para actualizaciones
    print(f"🔄 Procesando cambios híbridos en {target_table}")
    
    target = DeltaTable.forName(spark, target_table)
    
    merge_condition = " AND ".join([
        f"target.{k} = source.{k}" for k in business_key
    ]) + " AND target.es_actual = true"
    
    # Condición SCD2: hash de columnas SCD2 diferente
    scd2_change_condition = "target.hash_scd2 <> source.hash_scd2"
    
    # Condición SCD1: cualquier columna SCD1 diferente
    scd1_change_conditions = " OR ".join([
        f"target.{c} <> source.{c}" for c in scd1_columns
    ])
    
    source_prepared = source_with_hash.alias("source")
    
    # Ejecutar MERGE con ambas lógicas
    target.alias("target").merge(
        source_prepared,
        merge_condition
    ).whenMatchedUpdate(
        # SCD2: Cerrar registro cuando columnas importantes cambian
        condition=scd2_change_condition,
        set={
            "fecha_fin": current_timestamp(),
            "es_actual": lit(False),
            "fecha_actualizacion": current_timestamp()
        }
    ).whenMatchedUpdate(
        # SCD1: Sobrescribir columnas que cambian frecuentemente
        condition=f"NOT ({scd2_change_condition}) AND ({scd1_change_conditions})",
        set={
            **{c: col(f"source.{c}") for c in scd1_columns},
            "fecha_actualizacion": current_timestamp()
        }
    ).whenNotMatchedInsert(
        values={
            **{k: col(f"source.{k}") for k in business_key},
            **{c: col(f"source.{c}") for c in scd2_columns},
            **{c: col(f"source.{c}") for c in scd1_columns},
            **{c: col(f"source.{c}") for c in static_columns},
            "fecha_inicio": current_timestamp(),
            "fecha_fin": lit(None),
            "es_actual": lit(True),
            "version": lit(1),
            "hash_scd2": col("source.hash_scd2"),
            "fecha_creacion": current_timestamp(),
            "fecha_actualizacion": current_timestamp()
        }
    ).execute()
    
    print("✅ MERGE híbrido completado")

# Definir columnas para cada tipo
CUENTAS_SCD2_COLUMNS = ["tipo_cuenta", "limite_credito", "estado", "tasa_interes", "sucursal"]
CUENTAS_SCD1_COLUMNS = ["saldo_actual", "saldo_disponible", "fecha_ultimo_movimiento"]
CUENTAS_STATIC_COLUMNS = ["moneda", "fecha_apertura", "ejecutivo_cuenta"]

print("✅ Función apply_scd_hybrid definida")

In [ ]:
# ============================================
# CARGAR CUENTAS CON SCD HÍBRIDO
# ============================================

bronze_cuentas = spark.table(BRONZE_CUENTAS)

print(f"📊 Registros en Bronze Cuentas: {bronze_cuentas.count()}")

# Aplicar SCD híbrido
apply_scd_hybrid(
    source_df=bronze_cuentas,
    target_table=SILVER_DIM_CUENTAS,
    business_key=["numero_cuenta", "cliente_id"],
    scd1_columns=CUENTAS_SCD1_COLUMNS,
    scd2_columns=CUENTAS_SCD2_COLUMNS,
    static_columns=CUENTAS_STATIC_COLUMNS
)

In [ ]:
# Verificar resultado
print(f"\n📋 Muestra de {SILVER_DIM_CUENTAS}:")
spark.table(SILVER_DIM_CUENTAS).select(
    "cuenta_key",
    "numero_cuenta",
    "tipo_cuenta",
    "saldo_actual",
    "limite_credito",
    "estado",
    "es_actual",
    "version"
).show(10, truncate=False)

## 6️⃣ Queries Útiles para SCD

In [ ]:
# ============================================
# QUERY 1: Obtener Estado Actual de un Cliente
# ============================================

def get_current_record(table_name, business_key_col, key_value):
    """Obtiene el registro actual (vigente) de una dimensión SCD2"""
    return spark.table(table_name).filter(
        (col(business_key_col) == key_value) & 
        (col("es_actual") == True)
    )

print("📌 Estado actual de un cliente:")
get_current_record(SILVER_DIM_CLIENTES, "cliente_id", cliente_ejemplo).show(truncate=False)

In [ ]:
# ============================================
# QUERY 2: Obtener Estado a una Fecha Específica (Time Travel)
# ============================================

def get_record_as_of(table_name, business_key_col, key_value, as_of_date):
    """Obtiene el registro que era vigente en una fecha específica"""
    return spark.table(table_name).filter(
        (col(business_key_col) == key_value) &
        (col("fecha_inicio") <= as_of_date) &
        ((col("fecha_fin").isNull()) | (col("fecha_fin") > as_of_date))
    )

# Ejemplo: Estado hace 1 hora
from datetime import datetime, timedelta
fecha_consulta = datetime.now() - timedelta(hours=1)

print(f"📌 Estado del cliente a las {fecha_consulta}:")
get_record_as_of(
    SILVER_DIM_CLIENTES, 
    "cliente_id", 
    cliente_ejemplo, 
    fecha_consulta
).select("cliente_id", "nombre", "segmento_cliente", "ciudad", "version").show()

In [ ]:
# ============================================
# QUERY 3: Historial Completo de Cambios
# ============================================

def get_full_history(table_name, business_key_col, key_value):
    """Obtiene el historial completo de un registro"""
    return spark.table(table_name).filter(
        col(business_key_col) == key_value
    ).orderBy("version")

print(f"📜 Historial completo de {cliente_ejemplo}:")
get_full_history(SILVER_DIM_CLIENTES, "cliente_id", cliente_ejemplo).select(
    "version",
    "segmento_cliente",
    "ciudad",
    "direccion",
    "fecha_inicio",
    "fecha_fin",
    "es_actual"
).show(truncate=False)

In [ ]:
# ============================================
# QUERY 4: Estadísticas de SCD
# ============================================

print("📊 Estadísticas de SCD en Clientes:")

stats = spark.table(SILVER_DIM_CLIENTES).agg(
    count("*").alias("total_registros"),
    sum(when(col("es_actual") == True, 1).otherwise(0)).alias("registros_actuales"),
    sum(when(col("es_actual") == False, 1).otherwise(0)).alias("registros_historicos"),
    max("version").alias("max_versiones"),
    avg("version").alias("promedio_versiones")
)

stats.show()

# Distribución de versiones
print("\n📈 Distribución de versiones:")
spark.table(SILVER_DIM_CLIENTES).groupBy("version").count() \
    .orderBy("version").show()

## 7️⃣ Resumen y Mejores Prácticas

In [ ]:
# ============================================
# VERIFICACIÓN FINAL
# ============================================

print("\n" + "="*70)
print("📊 RESUMEN DE TABLAS SILVER")
print("="*70)

for table in [SILVER_DIM_CLIENTES, SILVER_DIM_CUENTAS]:
    df = spark.table(table)
    total = df.count()
    actuales = df.filter(col("es_actual") == True).count()
    historicos = total - actuales
    
    print(f"\n📁 {table}:")
    print(f"   └─ Total registros: {total:,}")
    print(f"   └─ Registros actuales: {actuales:,}")
    print(f"   └─ Registros históricos: {historicos:,}")
    print(f"   └─ Ratio histórico: {historicos/total*100:.1f}%" if total > 0 else "")

---

## ✅ Resumen del Notebook

### Lo que aprendimos:

1. ✅ **Tipos de SCD**: Type 0, 1, 2, 3 y cuándo usar cada uno
2. ✅ **SCD Type 2**: Implementación con historial completo
3. ✅ **SCD Híbrido**: Combinar Type 1 y Type 2 en la misma tabla
4. ✅ **Hash para detección de cambios**: Eficiencia en comparaciones
5. ✅ **Queries temporales**: Consultar estados históricos

### Columnas SCD Type 2:

| Columna | Tipo | Descripción |
|---------|------|-------------|
| `surrogate_key` | BIGINT | Clave sustituta única por versión |
| `business_key` | STRING | Clave natural del sistema origen |
| `fecha_inicio` | TIMESTAMP | Inicio de vigencia |
| `fecha_fin` | TIMESTAMP | Fin de vigencia (NULL = actual) |
| `es_actual` | BOOLEAN | Flag de registro vigente |
| `version` | INT | Número de versión |
| `hash_atributos` | STRING | Hash para detección de cambios |

### Mejores Prácticas:

- **Usar hash** para detectar cambios eficientemente
- **Separar columnas SCD1 y SCD2** en tablas híbridas
- **Índices en business_key + es_actual** para queries rápidos
- **Liquid Clustering** en business_key para optimización

### Próximo paso:
Continuar con el **Notebook 04: Gold Layer - Analytics** para crear métricas y KPIs.

---